In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

In [4]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data',
                                             train=True, 
                                             transform=transform,
                                             download=True)
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)



Files already downloaded and verified


In [5]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [7]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

total_step = len(train_loader)
curr_lr = learning_rate

In [8]:
# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.6077
Epoch [1/80], Step [200/500] Loss: 1.4756
Epoch [1/80], Step [300/500] Loss: 1.1409
Epoch [1/80], Step [400/500] Loss: 1.1469
Epoch [1/80], Step [500/500] Loss: 1.1465
Epoch [2/80], Step [100/500] Loss: 1.0018
Epoch [2/80], Step [200/500] Loss: 1.1671
Epoch [2/80], Step [300/500] Loss: 1.0152
Epoch [2/80], Step [400/500] Loss: 1.2360
Epoch [2/80], Step [500/500] Loss: 0.9691
Epoch [3/80], Step [100/500] Loss: 1.0306
Epoch [3/80], Step [200/500] Loss: 0.9567
Epoch [3/80], Step [300/500] Loss: 0.9065
Epoch [3/80], Step [400/500] Loss: 0.7631
Epoch [3/80], Step [500/500] Loss: 0.7854
Epoch [4/80], Step [100/500] Loss: 0.7708
Epoch [4/80], Step [200/500] Loss: 0.8028
Epoch [4/80], Step [300/500] Loss: 0.7222
Epoch [4/80], Step [400/500] Loss: 0.7621
Epoch [4/80], Step [500/500] Loss: 0.8701
Epoch [5/80], Step [100/500] Loss: 0.9875
Epoch [5/80], Step [200/500] Loss: 0.6229
Epoch [5/80], Step [300/500] Loss: 0.6369
Epoch [5/80], Step [400/500] Loss:

Epoch [39/80], Step [300/500] Loss: 0.2173
Epoch [39/80], Step [400/500] Loss: 0.1772
Epoch [39/80], Step [500/500] Loss: 0.2729
Epoch [40/80], Step [100/500] Loss: 0.1154
Epoch [40/80], Step [200/500] Loss: 0.3290
Epoch [40/80], Step [300/500] Loss: 0.1919
Epoch [40/80], Step [400/500] Loss: 0.1453
Epoch [40/80], Step [500/500] Loss: 0.1545
Epoch [41/80], Step [100/500] Loss: 0.3003
Epoch [41/80], Step [200/500] Loss: 0.1491
Epoch [41/80], Step [300/500] Loss: 0.2155
Epoch [41/80], Step [400/500] Loss: 0.2909
Epoch [41/80], Step [500/500] Loss: 0.1791
Epoch [42/80], Step [100/500] Loss: 0.2934
Epoch [42/80], Step [200/500] Loss: 0.1806
Epoch [42/80], Step [300/500] Loss: 0.2297
Epoch [42/80], Step [400/500] Loss: 0.2200
Epoch [42/80], Step [500/500] Loss: 0.1350
Epoch [43/80], Step [100/500] Loss: 0.0783
Epoch [43/80], Step [200/500] Loss: 0.1686
Epoch [43/80], Step [300/500] Loss: 0.1694
Epoch [43/80], Step [400/500] Loss: 0.1265
Epoch [43/80], Step [500/500] Loss: 0.1468
Epoch [44/8

Epoch [77/80], Step [400/500] Loss: 0.1351
Epoch [77/80], Step [500/500] Loss: 0.1608
Epoch [78/80], Step [100/500] Loss: 0.2434
Epoch [78/80], Step [200/500] Loss: 0.1808
Epoch [78/80], Step [300/500] Loss: 0.0858
Epoch [78/80], Step [400/500] Loss: 0.1004
Epoch [78/80], Step [500/500] Loss: 0.1002
Epoch [79/80], Step [100/500] Loss: 0.1227
Epoch [79/80], Step [200/500] Loss: 0.0848
Epoch [79/80], Step [300/500] Loss: 0.0593
Epoch [79/80], Step [400/500] Loss: 0.1451
Epoch [79/80], Step [500/500] Loss: 0.0511
Epoch [80/80], Step [100/500] Loss: 0.0947
Epoch [80/80], Step [200/500] Loss: 0.0717
Epoch [80/80], Step [300/500] Loss: 0.1784
Epoch [80/80], Step [400/500] Loss: 0.1826
Epoch [80/80], Step [500/500] Loss: 0.1684


In [ ]:
test_dataset = torchvision.datasets.ImageFolder('./img_gen', transform=transforms.Compose([transforms.Scale((32,32)),
                                                                                           transforms.ToTensor()]))
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('GAN Test score: {} %'.format(correct / total))